## PROBLEM DESCRIPTION

The data we want to analyze is styles from a group of elements. In this case we had 4 styles so we want at least one machine for every one that it will be able to analyze the data an generate new data.

So the data that we have is a set of vectors that's the output of the app that generate grids. This grids ara a composition of elements so the outuput will be a vector of id. Each grid had 48 elements and each element can have a maximum of 255 elements in it. But is not necessary to had that number, it can be possible to have less. 

<h4 align="center">$[ x_1, x_2, x_3, ..., x_{48} ]$</h4> 
<h4 align="center">$x_1 = [id_1,id_2,_id_3, ...,id_{255}]$</h4>

Each element of the vector can have a different size from the others and the list of integers are the subcategories of style of the object that was placed in the grid. Each number is a different subcategory of style of the object placed previously.

So the first objective is to descompose the input data, a vector which element is a vector of different size, to a list of vectors that will have de same length, 48, and one element in each position. So the main goal is to find the grid wich elements has more IDs so we can set the size of total vectors,$y$, that we need to descompose the data. In case that one element of the vector has lower IDs that the max we gonna put a $0$ as the null value for our data.

<h4 align="center">$input = [[id_1^1,id_2^1,_id_3^1, ...,id_{255}^1], [id_1^2,id_2^2,_id_3^2, ...,id_{255}^2], ..., [id_1^{48},id_2^{48},_id_3^{48}, ...,id_{255}^{48}]]$</h4> 
<h4 align="center">$output = [[id_1^1, id_1^2,..., id_1^{48}],
[id_2^1, id_2^2,..., id_2^{48}],..., [0,0,...,id_y^{48}]$</h4> 

In [1]:
import pandas as pd
import json
import sys
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from timeit import default_timer as timer
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
import random
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM
import Utils

In the file *Utils.py* we have 2 two functions that allow us to transform our input data. 

The first one is *getMaxLengthForElements*. This functions returns a integer that specifies the max length that the vectors must have after analyzing all the input vectors.


The second one is *transformInputVector*. This function transform the input data by transforming each vector to the maximum length and adding $0$ to the vectors who had less length than the maximum.


Applying this 2 functions we can transform our data to be processed and converted to one hot vectors.

In [3]:
# vector_data = pd.read_json('./Data.json')
vector_data = pd.read_json('./Transformed_Moodboards.json')
A = vector_data.values
data_modern = []
A = A.ravel()
length_vectors = len(A)
for i in range(A.shape[0]):
    len_v = len(A[i])
    tmp = []
    for j in range(len_v):
        tmp.append(A[i][j])
    tmp = np.asarray(tmp).ravel()
    data_modern.append(tmp)
data_modern = np.asarray(data_modern)
input_data = []
input_data.append(data_modern)
input_data = np.asarray(input_data)
max_elements = Utils.getMaxLengthForElements(input_data[0])
f_data = Utils.transformInputVector(input_data[0], max_elements, length_vectors)

#Multiply data for obtain more range <= 10000.
for i in range(50):
    for k in range((length_vectors*max_elements)):
        f_data.append(f_data[k])
f_data = np.asarray(f_data)
print(max_elements)
print(f_data[2])
print(f_data.shape)

15
[ 25   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  25  25  25 178  24  56  25  25  25  25  46
  56  25  99   0  25  26  25   0   0   0  25  25]
(6120, 48)


At this poing we have the data formatted ready for the input for the machine. In this case we gonna use the algorithm of the library sklearn, BernoulliRBM. Also we gonna use the implementation of One Hot Vector to transform our data to One Hot Vectors.

http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Now the first step to get the data in the machine is to convert all the data we formatted to one hot vectors. A one hot vector is a vector which is is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

In [5]:
%%time
oneHotEncoder = OneHotEncoder(262, sparse=False).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)
print(oneHotEncoder)
print(oneHotData.shape)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values=262, sparse=False)
(6120, 12576)
Wall time: 341 ms


In [6]:
%%time
oneHotEncoder = OneHotEncoder(262, sparse=True).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)
print(oneHotEncoder)
print(oneHotData.shape)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values=262, sparse=True)
(6120, 12576)
Wall time: 59.8 ms


At this point we have 3 hot vectors which one represents the previous input data. This is a sparse matrix so is fastest to work than a simple vector and it's easy to show the results.
Then we neeed to define the parameters of the machine and fit it with de one hot vectors we had. The parameters are not specified, so we need to try whichc are the best to obtain good results


In [8]:
RBM_Machine = BernoulliRBM(n_components=262, learning_rate=0.01, n_iter=20, random_state=0, verbose=True)
print(RBM_Machine)

BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=262, n_iter=20,
       random_state=0, verbose=True)


In [4]:
%%time
RBM_Machine = BernoulliRBM(n_components=262, learning_rate=0.01, n_iter=20, random_state=0, verbose=True)
# RBM_Machine = BernoulliRBM(n_components=262, learning_rate=0.01,batch_size=64, n_iter=20, random_state=0, verbose=True)
RBM_Machine.fit(oneHotData)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -1018.87, time = 53.73s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -78.52, time = 57.54s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -80.80, time = 60.48s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -74.14, time = 55.01s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -77.79, time = 52.40s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -75.31, time = 53.13s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -74.91, time = 52.39s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -77.24, time = 52.09s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -75.42, time = 52.31s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -75.18, time = 56.31s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -81.44, time = 61.02s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -73.93, time = 55.00s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -71.03, time = 66.80s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -79.37, time = 61.06s
[BernoulliRBM

Now we gonna save the RBM and one hot data so we can use it later.

In [5]:
import pickle

filename = 'RBM.pickle'
pickle.dump(RBM_Machine, open(filename, 'wb'))
pickle_out = open("OneHotData.pickle","wb")
pickle.dump(oneHotData, pickle_out)
pickle_out.close()

Finally we got the output. We gonna make as much as the maximum length of samples. Then we gonna format this data and save it to a json in our folder.

In [6]:
output_grid = Utils.generateGrid(1)

In [7]:
print(output_grid)

[[[4, 5, 8, 24, 26, 79, 91, 97], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [5, 24, 42, 88, 140, 178], [2, 24, 25, 138, 178, 239], [40, 56, 84], [39], [5, 24, 25, 27, 46, 51, 56, 88, 178, 234], [4, 154, 155], [178], [24, 46, 178], [5, 24, 247], [9, 178, 220], [9, 17, 223], [46, 218, 223], [5, 10, 233], [5, 24, 97], [25], [3, 25, 212, 223, 233], [17, 24, 25, 27, 233], [12, 14, 22, 23, 24, 239], [5], [5, 241], [241], [25], [53, 98]]]


In [ ]:
# import codecs, json 
# b = final_output_grid
# json_file = "./output_grid.json" 
# json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

## OTHER IMPLEMENTATION

In [ ]:
oneHotEncoder = OneHotEncoder(262, sparse=False).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)
test_Data_Vector_Aux = np.array(oneHotData, dtype="float64")

In [ ]:
visible_dim = oneHotData.shape[1]
hidden_dim = 500
epochs = 100
K = 1
lr = 0.1
batch_size = 10

In [ ]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)

In [ ]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=1,
         lr=0.01,
         epochs=20,
         batch_size=64,
         plot_weights=False)

In [ ]:
final_output = []
for i in range(100):
    x_visible, x_h = rbm_.sample_visible_from_visible(oneHotData[random.randint(0,oneHotData.shape[0])])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

In [ ]:
final_output_grid = Utils.cleaningOutput(output_grid)
print(final_output_grid)

In [ ]:
b = output_grid
json_file = "./output_grid_2.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

###  Evaluation Quantitativa

Evaluar models generatius és en general molt complicat.

Per a decidir que el nostre sistema ¨va bé¨ (va millor que generar coses aleatoriament). S´ha decidit generar

- 25 dades amb la RBM. 
- 25 dades aleatories.

Donem les graelles al expert i li preguntem quines son bones i quines no. Contem quantes "de les bones" corresponnen amb les generades per la RBM.

#### Evaluacio custom

Podriem mirar de les graelles generades si els objectes son de color similar.

###  Evaluation Qualitativa

Posar en el document les graelles generades per la RBM, alguna graella random i alguna graella original. Explicar quins aspectes son bons i quins dolents de les graelles generades. Per exemple, demanar al expert que, de les graelles RBM generades quines coses 'no li agraden'.

### Posar imatge de la eina

Interficie grafica de la imatge.

- Que ap
orta la eina
- Temps que triguen els humans a generar una graella amb photoshop
- Temps que triguen els humans a generar una graella amb eina
- Temps que triguen els humans a generar una graella amb RBM
    - Podries vendre-ho com una "help tool" que a un disenyador li genera 25 graelles i ell sols ha de triar
      quina li agrada més. I si vol variar alguna cosa de la graella.
    
